# Lab 5 - Localization
## Introduction
(*Open this notebook in Chrome or Firefox to avoid incorrect equation rendering in Safari*)

Determining a robot’s orientation and position in a known environment, also known as localization, is a critical problem in the field of robotics. As is common in robotics, this seemingly simple problem is surprisingly difficult, and remains an active research area. In this lab, you will solve robotic localization by implementing Monte Carlo Localization (aka MCL or particle filter).

This lab consists of three parts, the first two of which must be submitted individually:

-   **Part A** - (Writing Assignment) Understand the motion and sensor model.
-   **Part B** - (Programming Assignment) Develop and test the particle filter algorithm in simulation.  
-   **Part C** - Demonstrate the functionality of your algorithm on the real RACECAR. 
-   **Part D** - (**OPTIONAL**) From localization to SLAM: Configure and run **Google Cartographer** on the real RACECAR.

<img src="figures/pf.png" width="600">

## Submission 
<a id='submission'></a>
    **Part A** - (**INDIVIDUAL EFFORT**) Submit your answers **INDIVIDUALLY** to the writing assignment on **gradescope**, preferably $\LaTeX$ed. You must show work (derivations, formulas used, etc.) to receive full credit. You will also need to put your numeric results in `autograder/solutions_go_here.py` and upload to gradscope. You may collaborate on problems but you cannot simply share answers. You must write up your solutions independently.

Parts A is due at **1 pm, Wednesday, March 20, 2019**, which is 1 week after the release date of this lab.

**Part B** - (**TEAMWORK**) Submit your `log.npz` autograder output on **gradescope**. After running the automated tests for your implementation of MCL, the autograder will generate a file called  `log.npz`.  **You must upload this to gradescope to receive credit for part B**.

**Part C** - (**TEAMWORK**) Run your implementation of MCL from part B on the real RACECAR and present your results to the teaching staff. The things we will look into:
   -   Numerical evidence that your algorithm is working in the form of charts/data
		-   Convergence rates, cross track error, etc
		-   The simulator odometry is perfectly accurate, so that can be your ground truth! Show how the robust the simulator is in the presence of noisy odometry.
   -   An [illustrative video](https://www.youtube.com/watch?v=-c_0hSjgLYw&t=6s) of your particle filter working in a simulation environment, overlaying:
		-  Visualization of inferred position
		-   Visualization of the particle distribution
		-   The known map
		-   Laser scan data in the coordinate frame of your inferred position (it should align fairly well with the walls in the known map)
		-   Any other byproducts of your algorithm which you find worthy of visualization
   -   A [similarly illustrative video of your car localizing](https://www.youtube.com/watch?v=-c_0hSjgLYw) in the Stata basement environment

**Part D** (**TEAMWORK**, *OPTIONAL*) Configure and run Google Cartographer on the real RACECAR.
- Build a map of the Stata basement environment using Cartographer
- Localize using Cartographer
- Show visualization evidence that Cartographer is working on the RACECAR

Parts B & C & D are due at **1 pm Wednesday, April 3, 2019**, which is 2 weeks after the release date of this lab.
This lab will **be presented alongside lab 6**.



## Grading (10+2 points)
**Part A**. Writing Assignment (3 points)
- 1 point for part (i) in question 1 
- 1 point for part (ii) in question 1
- 1 point for question 2 

**Part B**. Programming implementation (4 points)

**Part C**. Lab report, presentation (3 points)

**Part D** (**Bonus**). Lab report together with part C, and presentation (2 points)

## Part A - Writing Assignment
Work on the following questions, submit your numeric answers to autograder, and justify your answers in your lab report. Just writing the final answer will not give total credit, even if correct. These questions will help you understand the algorithm before diving into coding.

<a id='question1'></a>
**Question 1**. (**Motion Model**) Consider a deterministic motion model (no added noise) based on odometry information. The motion model, $g$, takes as arguments the old particle pose, $\mathbf{x}_{t-1}$, as well as the current odometry data, $\Delta\mathbf{x}$, and returns a new pose, $\mathbf{x}_t$ with the odometry “applied” to the old poses:

$$
\begin{align}
    \mathbf{x}_t = \begin{bmatrix}x_t\\y_t\\\theta_t\end{bmatrix} = g(\mathbf{x}_{t-1}, \Delta x)
\end{align}
$$

Note that the poses $\mathbf{x}$ are expressed in the world frame relative to some arbitrary origin. The odometry $\Delta \mathbf{x}$ is necessarily a local quantity, expressed relative to the previous frame.

**i** Suppose you are given $\mathbf{x}_{t-1} = \left[0, 0, \frac{\pi}{6}\right]^T$ and $\mathbf{x}_{t} = \left[0.2, 0.1, \frac{11\pi}{60}\right]^T$. Compute the odometry data $\Delta \mathbf{x}$ that results in this transformation.

**ii**. Now suppose you received the odometry data $\Delta\mathbf{x}$ from part **i**, but your car was previously at position $\mathbf{x}_{t-1} = \left[3, 4, \frac{\pi}{3}\right]^T$. Compute the current pose $\mathbf{x}_t$.

If you were to use this deterministic motion model in your particle filter all of your particles would end up in the same place - which defeats the purpose of having so many. When you build your actual motion model, you will be injecting noise into the function $g$ which will make your particles spread out as the car moves. This accounts for any uncertainty that exists in the odometry information. The sensor model will collapse this spreading of particles back into a small region.

<a id='question2'></a>
**Question 2**. (**Sensor Model**) The sensor model, $p\left(z_{t}| x_{t}, m\right)$, defines how likely it is to record a given sensor reading $z_{t}$ from a hypothesis position $x_{t}$ in a known, static map $m$ at time $t$. We will use this likelihood to "prune" the particles as the motion model tries to spread them out. Particles will be kept if the sensor reading is consistent with the map from the point of view of that particle. The definition of this likelihood is strongly dependent on the type of sensor used - a laser scanner in our case.

In a lidar model, typically, there are a few cases to be modeled in determining $p\left(z_{t}| x_{t}, m\right)$:

1. Probability of detecting a known obstacle in the map
2. Probability of a short measurement. Maybe due to internal lidar reflections (scratches or oils on the surface), hitting parts of the vehicle itself, or other unknown obstacles (people, cats, etc).
3. Probability of a very large (aka missed) measurement. Usually due to lidar beams that hit an object with strange reflective properties and did not bounce back to the sensor
4. Probability of a completely random measurement. Just in case of an asteroid strike.
    
We typically represent (1) with a gaussian distribution centered around the ground truth distance between the hypothesis pose and the nearest map obstacle. Thus, if the measured range exactly matches the expected range, the probability is maximum. If the measured range is $z_{t}$ and the ground truth range is determined (via ray casting on the map $m$ from pose $x_t$) to be $z_{t}^*$ then we have that:

$$
	p_{hit}(z_{t}| x_{t}, m)  = \begin{cases}
	\eta \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(z_t - z_t^*)^2}{2\sigma^2}\right)  &   \text{if} \quad 0 \leq z_{t}^{k} \leq z_{max}\\
    0   &   \text{otherwise}
    \end{cases}
$$
where $\eta$ is a normalization constant need to be determined.

Case (2) is represented as a downward sloping line as the ray gets further from the robot.
This is because if the unknown obstacles (people cats, etc.) are distributed uniformly in the environment, the lidar is more likely to hit ones that are closer (think about how area scales...). This likelihood can be modeled as:

$$
	p_{short}\left(z_{t}| x_{t}, m\right) =  \frac{2}{z_t^*} \begin{cases}
         1 - \frac{z_{t}}{z_{t}^*}   &   \text{if} \quad 0 \leq z_{t}^{k} \leq z_{t}^*\\
         0   &   \text{otherwise}
\end{cases}
$$

Case (3) is represented by a large spike in probability at the maximal range value, so that reflected measurements do not significantly discount particle weights.

$$
	p_{max}(z_{t}| x_{t}, m)  =\begin{cases}
	1  &  \text{if} \quad z_t = z_{max}\\
	0  &  \text{otherwise} 
	\end{cases}
	$$

Case (4) is represented by a small uniform value, to account for unforeseen effects.

$$
	p_{rand}(z_{t}| x_{t}, m)  = \begin{cases}
	\frac{1}{z_{max}}  &  \text{if} \quad 0\leq z_{t} \leq z_{max}\\
	0                            & \text{otherwise} 
	\end{cases}
$$
	
These four different distributions are now mixed by a weighted average, defined by the parameters $\alpha_{hit}, \alpha_{short},\alpha_{max},\alpha_{rand}$:

$$
	 p(z_{t}| x_{t}, m)  = \alpha_{hit} \cdot p_{hit}(z_{t}| x_{t}, m)  + \alpha_{short} \cdot p_{short}(z_{t}| x_{t}, m)  + \alpha_{max} \cdot p_{max}(z_{t}| x_{t}, m)  + \alpha_{rand} \cdot p_{rand}(z_{t}| x_{t}, m) 
$$

Note that in order for $p(z_{t}| x_{t}, m)$ to be a probability distrubution we have have that:

$$
\alpha_{hit}+\alpha_{short}+\alpha_{max}+\alpha_{rand}=1
$$

All together this should look like this:

<img src="figures/sensor_model_slice2.png" width="600">
	 
Find the values of $p(z_{t} | x_{t}, m)$ for the values of $z_t$ below and the following parameters:
$\alpha_{hit} = 0.74$,
$\alpha_{short}=0.07$,
$\alpha_{max}=0.07$,
$\alpha_{rand}=0.12$,
$\sigma=0.5\text{ m}$
$z_{max}=10\text{ m}$, and
$z_{t}^{*} = 7\text{ m}$.

**i**.    $z_{t} = 0\text{ m}$

**ii**.   $z_{t} = 3\text{ m}$

**iii**.  $z_{t} = 5\text{ m}$

**iv**.   $z_{t} = 8\text{ m}$

**v**.    $z_{t} = 10\text{ m}$

## Part B - Programming Assignment

### 1. Getting Started

Grab the skeleton code from this repository and stick it in your ```src``` folder.

#### 1.1 Installing Dependencies

We are helping you out a little bit by taking care of ray tracing for you. This step needs to be repeated many many times and it can be quite slow without the help of C++. Plus it happens to be the same exact code used to generate the simulated scan in the racecar simulator!

First update your simulator (we made some small optimization changes..)

    cd ~/racecar_ws/src/racecar_simulator
    git pull

Then run ```catkin_make``` **and** ```catkin_make install```

    cd ~/racecar_ws
    catkin_make
    catkin_make install
    
All updated! Now you just need to build the code for the lab. We're doing this in place:

    sudo apt-get install cython
    cd ~/racecar_ws/src/localization/src
    python setup.py build_ext --inplace

### 2. Implementation


#### 2.1 Motion Model

Implement your motion model in the ```motion_model.py``` file.

While you could use the IMU in the motion model, we recommend using the wheel odometry coming from dead-reckoning integration of motor and steering commands. It will probably be less noisy than the IMU except in extreme operating conditions. You may assume that the motor commands relate to the movement of your car according to the [geometric Ackermann model](http://www.ri.cmu.edu/pub_files/2009/2/Automatic_Steering_Methods_for_Autonomous_Automobile_Path_Tracking.pdf), with some noise due to tire slip and other imperfections. The `/vesc/odom` topic does this for you already on the car (```/odom``` in the simulator)!

The precise definition of your motion model is up to you. The main constraints are:

- You must add random noise! Otherwise, as mentioned earlier, the particles won't spread out.
- As the noise approaches zero, the motion model should become exactly equal to the deterministic model you worked with in the writing section.

You might empirically determine your noise coefficients based on what works, or could try to gather data from the car which allows you to directly determine your measurement uncertainty. You will also have to model how your available odometry data (i.e. steering commands speed and steering) correspond to changes in states $(x,y,\theta)$.

We recommend you start with the formulas you derived in [Question 1 of part A](#question1) and think of ways to add noise to these equations in a way that makes sense geometrically. The algorithm ```sample_motion_model_odometry``` in the [Probabilistic Robotics textbook](https://docs.ufpr.br/~danielsantos/ProbabilisticRobotics.pdf) may also help although it is not necessarily accurate to the car's geometry. All of Chapter 5 is a great resource to understand this lab's theory a little more.

#### 2.2 Sensor Model

Implement your sensor model in the ```sensor_model.py``` file.

Once you have updated particle positions via your motion model, you use the sensor model to assign likelihood weights to each particle. This allows good hypotheses to have better likelihood of being sampled on the next iteration, and visa versa for bad hypotheses. 


#### Precomputing the model

You may have noticed while doing the writing section that the sensor model is quite complicated and requires a lot of multiplications and non-trivial functions. And in the end this function is something that is "guessed". It will never reflect the true distribution and so long as it has the general shape that is good enough. So rather than evaluating this function millions of times at runtime, you can evaluate it on a discretized grid of inputs $z_t$ and $z_t^*$ and store these values in a lookup table.

This strategy is described in section 3.4.1 of [2]. The motivation is twofold:

1. Reading off a probability value from a table based on your actual sensor measurement and raycast ‘ground truth’ values is **MUCH** faster than doing all the math every time;
2. This provides a good opportunity to numerically normalize the $p(z_{t} |x_t, m)$ function (make sure your probabilities sum to 1).

If you plot your surface you should get something like this:

<img src="figures/sensor_model2.png" width="600">


#### Ray casting

As we mentioned, we are doing ray casting for you. If you pass an array of particles (and $n\times 3$ matrix) into our ray tracer you will get an $n\times m$ matrix where $m$ is the number of lidar beams. This is essentially a stack of $n$ lidar messages which you are probably very familiar with at this point. There is one lidar message for each particle, which is the scan that would be seen from that point.

For example if you wanted to call it on a bunch of particles that are all $(0, 0, 0)$ you could do:

    poses = np.zeros((n, 3))
    scans = self.scan_sim.scan(poses)

#### 2.3 Putting it all together: The particle filter

Once you have a motion model and a sensor model you can use them to construct the complete MCL algorithm. Do this in the ```particle_filter.py``` file.

From a very high level this will work as follows:

- Whenever you get odometry data use the motion model to update the particle positions
- Whenever you get sensor data use the sensor model to compute the particle probabilities. Then resample the particles based on these probabilities

You will also consider how you want to initialize your particles. We recommend that you should be able to use some of the interactive topics in rviz to set an initialize "guess" of the robot's location. Global localization (aka the [kidnapped robot problem](https://en.wikipedia.org/wiki/Kidnapped_robot_problem)) is very hard.

You may find the [numpy.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) function useful for resampling!

#### Debugging

The best way to debug the particle filter is by isolation. First, turn off your sensor model (don't ever resample). If your particles start out in a small location, they will begin to move together and slowly spread out as the travel distance increases. Remember in the absense of noise the motion model should exactly mimic the motion of the car.

Make some test cases for the sensor model. For example scans that are identical should have a very high probability. Scans that are completely different should have a very low probability. What other useful cases could you write tests for? Visualizing is super helpful. Use the [PoseArray message](http://docs.ros.org/lunar/api/geometry_msgs/html/msg/PoseArray.html)!!!!

### 3. **Test your implementation in simulation**.
 Similar to previous labs, we have provided an autograder to test your particle filter. The autograder will launch the following nodes:
-       `racecar/launch/teleop.launch` (temporarily at startup)
-   	`localization/launch/map_server.launch`
-   	`localization/launch/localize.launch`
    

Once the autograder detects your particle filter is running (as soon as it receives a message from `/pf/pose/odom`) it will launch a rosbag collected on a Hokuyo flavored RACECAR while driving around the basement. Then, the autograder receives messages published on `/pf/pose/odom` and extracts (time, x, y, theta) information from each message via `Odometry.pose.pose.position`, `quaternion_to_angle` (`Odometry.pose.pose.orientation`) and `Odometry.header.stamp`. When the test completes (or is terminated early) the autograder will output a `log.npz` file that contains this trajectory data, which you need to upload to Gradescope. You should all submit the log.npz file, but you can share that file among your own team.

Periodically throughout testing, we provide initialization via the `/initialpose `topic, so it’s extremely important that you implement this functionality in your solution. You are not allowed to manually provide your own reinitialization messages during testing.

You may run the test at less than real time via the `--rate` argument (e.g. `./run_tests --rate 0.5`). However, the grade you receive will also be scaled by this quantity according to the below chart, so it is in your interest to run at real time! For fairness, we require that all teams run the test scripts on their car. This will avoid potentially penalizing people with slower laptops.

FYI, TA's solution generally gets $>0.9$ points on Gradescope. Note: the bit where the car is by the bikes is tricky! The map isn’t perfect, and therefore you will not achieve perfection there. Don’t worry about it, everyone will have the same problem.

Note: you will have to install the map server node on the RACECAR.

	sudo apt-get install ros-kinetic-map-server
    
<img src="figures/scaling.png" width="600">

### 4. Tips and Tricks

Since the algorithm must run at >20Hz with a large number of particles, an efficient implementation is a requirement for success. There are a few tricks you can use, primarily:

- Downsample your laser scan: you lidar has > 1000 beams but many of them are redundant. Downsample to ~100 for good performance. This will make the probability distribution over your state space less "peaked" and increase the number of particles you can maintain in real time (less ray casting).

- Also to make your probability distribution less peaked you could also try to "squash" your sensor model output probability by raising it to a power of less than one (1/3 for example) to make your distribution even less peaked. If your are confused by this paragraph, look at [4,5]. 

- Don't go crazy with particles: start with ~200. You can probably get your code running with thousands of particles but it will take some well crafted code to run in real time.

- Use ```rostopic hz``` to check the rate at which you are publishing control messages. It should be greater than 20hz for realtime performance.

- Use numpy arrays for absolutely everything - python lists → slow
    - Use numpy functions on numpy arrays to do any computations.
    - avoid Python for loops like the plague
    - [Slice indexing is your (best) friend.](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)
    
-   Use the smallest number of operations required to perform your arithmetic

-   Avoid unnecessary memory allocations
    -   Cache and reuse important numpy arrays by setting them to the right size during initialization of your particle filter as “self” variables
    
-   Identify your critical code paths, and keep them clean
    - Conversely, don’t worry too much about code that is called infrequently
    
-   Push code to Cython/C++ if necessary
    -   But don't worry about this too much - we already did this for the ray tracing part.
    
-   Avoid excessive function calls - function call overhead in Python → slow

-   Don’t publish visualization messages unless someone is subscribed to those topics
```
    if  self.[some_pub].get_num_connections() >  0 …
```

-   USE A PROFILER to identify good candidates for optimization

## Part C - Run on Real RACECAR
1. Demonstrate the functionality of your algorithm on the real RACECAR; 

3. Check [Submission](#submission) for further instruction.

We will announce the schedule for  the presentation in next lab.

## Part D - (OPTIONAL) Simultaneous Localization and Mapping - Configure and Run Google Cartographer
Configure and run **Google Cartographer** on the real RACECAR. [Cartographer](https://github.com/googlecartographer/cartographer) is a system that provides real-time simultaneous localization and mapping ([SLAM](https://en.wikipedia.org/wiki/Simultaneous_localization_and_mapping)) in 2D and 3D across multiple platforms and sensor configurations.  Please go to [this repository](https://github.com/mit-rss/cartographer_config), follow the instruction to configure and run cartographer on the real RACECAR.

## References

1. <a name="ThrunRobust"></a>[S. Thrun, D. Fox, W. Burgard and F. Dellaert. “Robust Monte Carlo Localization for Mobile Robots.” Artificial Intelligence Journal. 2001](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.71.6016&rep=rep1&type=pdf).
2. <a name="FoxMarkov"></a>[D. Fox, W. Burgard, and S. Thrun. “Markov localization for mobile robots in dynamic environments,” Journal of Artificial Intelligence Research, vol. 11, pp. 391427, 1999](http://www.jair.org/media/616/live-616-1819-jair.pdf).
3. <a name="FoxAdvances"></a>[D. Fox. “KLD-sampling: Adaptive particle filters,” Advances in Neural Information Processing Systems 14 (NIPS), Cambridge, MA, 2002. MIT Press](https://papers.nips.cc/paper/1998-kld-sampling-adaptive-particle-filters.pdf).
4. <a name="BagnellPracticle"></a> [D. Bagnell “Particle Filters: The Good, The Bad, The Ugly”
](http://www.cs.cmu.edu/~16831-f12/notes/F14/16831_lecture05_gseyfarth_zbatts.pdf)
5. <a name="BootsImportance"></a>[B. Boots “Importance Sampling, Particle Filters”](https://web.archive.org/web/20170209092754/http://www.cc.gatech.edu/~bboots3/STR-Spring2017/Lectures/Lecture4/Lecture4_notes.pdf)
6. <a name="WalshCDDT"></a>[C. Walsh and S. Karaman “CDDT: Fast Approximate 2D Ray Casting for Accelerated Localization”](https://arxiv.org/abs/1705.01167)